In [ ]:
%%capture
!pip install datasets
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

# Load Conceputal Captions

In [ ]:
# Specify dataset set sizes
train_size = 100000
val_size = 1000
dataset_train, dataset_val = load_dataset("conceptual_captions", split=[f"train[:{train_size}]", f"validation[:{val_size}]"])

In [ ]:
batch_size = 1000

# Helpers

In [ ]:
# Check fetch rate
def print_summary(dataset, key, action = "Fetched"):
    img_count_w_embedding = sum(1 for entry in dataset if entry[key] is not None)
    total_img_count = len(dataset)

    percentage_embedded = (img_count_w_embedding / total_img_count * 100) if total_img_count > 0 else 0

    print(f"{action} {img_count_w_embedding} out of {total_img_count} images ({percentage_embedded:.2f}%)")

## Fetch Image Helpers

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter

In [ ]:
# Maintain session
session = requests.Session()
adapter = HTTPAdapter(pool_connections=100, pool_maxsize=100)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [ ]:
from datasets.utils.file_utils import get_datasets_user_agent

USER_AGENT = get_datasets_user_agent()

In [ ]:
def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            response = session.get(
                image_url, headers={"user-agent": USER_AGENT},
                timeout=timeout
            )
            response.raise_for_status()
            image = Image.open(BytesIO(response.content))
            return image
        except Exception as e:
            return None

def fetch_images(batch, num_threads, timeout=None, retries=0):
    fetch_image_with_args = partial(
        fetch_single_image,
        timeout=timeout,
        retries=retries
    )
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image"] = list(
            executor.map(
                fetch_image_with_args,
                batch["image_url"]
              )
            )
    return batch

## Embed Image Helpers

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoImageProcessor, BeitModel, ViTModel

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
import os
from datasets import Dataset

def embed_single_image(image, image_processor, model, device):
    try:
        inputs = image_processor(image, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[0, 0, :].cpu()
        avg_embedding = outputs.last_hidden_state.squeeze().mean(axis=0)
        #return cls_embedding
        return avg_embedding
    except Exception as e:
        return None


def embed_images_and_save(batch, image_processor, model, device):
    image_urls = []
    captions = []
    embeddings = []

    for item in batch:
        image_url = item['image_url']
        caption = item['caption']
        image = item['image']

        embedding = None
        if image is not None:
            embedding = embed_single_image(image, image_processor, model, device)

        image_urls.append(image_url)
        captions.append(caption)
        embeddings.append(embedding)

    # Convert lists to a HuggingFace Dataset
    dataset = Dataset.from_dict({
        'image_url': image_urls,
        'caption': captions,
        'image_embedding': embeddings
    })

    return dataset

# Process Data

In [ ]:
drive_folder = "MIT 6.8610"

In [ ]:
num_threads = 40
fetch_fn_kwargs = {
    "num_threads": num_threads,
    "timeout": 5,
    "retries": 1
}

if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)
else:
    device = torch.device("cpu")

In [ ]:
import math
from datasets import load_from_disk, concatenate_datasets
import time
import shutil
from tqdm import tqdm

def process_and_save_dataset(
    dataset,
    drive_base_folder,
    batch_folder,
    agg_folder,
    file_name_base,
    image_processor,
    model,
    device,
    batch_size=batch_size,
    fetch_fn_kwargs=fetch_fn_kwargs,
):
    total_items = len(dataset)
    num_batches = math.ceil(total_items / batch_size)

    folder_path_batches = os.path.join(
          '/content/drive/My Drive',
          f"{drive_base_folder}/{batch_folder}"
    )

    def get_batch_file_path(batch_num):
      file_name = f"{file_name_base}_batch_{batch_num}.hf"
      return os.path.join(folder_path_batches, file_name)

    for i in tqdm(range(num_batches), desc="Processing batches"):
        # Process data in batches
        start_idx = i * batch_size
        end_idx = min(start_idx + batch_size, total_items)
        indices = range(start_idx, end_idx)
        batch = dataset.select(indices)

        print(f"Fetching images for batch {i+1}/{num_batches}...")
        batch_with_images = batch.map(
            fetch_images,
            batched=True,
            batch_size=100,
            fn_kwargs=fetch_fn_kwargs
        )
        print_summary(batch_with_images, key="image", action="Fetched")

        print(f"Embedding images for batch {i+1}/{num_batches}...")
        batch_with_embeddings = embed_images_and_save(
            batch_with_images,
            image_processor,
            model,
            device
        )

        full_file_path_batch = get_batch_file_path(i)
        batch_with_embeddings.save_to_disk(full_file_path_batch)

    # Aggregating datasets
    datasets = []

    for batch_num in range(num_batches):
        full_file_path_batch = get_batch_file_path(batch_num)
        batch_data = load_from_disk(full_file_path_batch)
        datasets.append(batch_data)

    aggregated_dataset = concatenate_datasets(datasets)
    # aggregated_dataset = aggregated_dataset.filter(lambda item: item["image_embedding"] is not None)

    ts = time.time()
    file_name_agg_dataset = f"{file_name_base}_{total_items}_{ts}.hf"
    folder_path_agg = os.path.join(
        '/content/drive/My Drive',
        f"{drive_base_folder}/{agg_folder}"
    )
    full_file_path = os.path.join(folder_path_agg, file_name_agg_dataset)

    aggregated_dataset.save_to_disk(full_file_path)

In [ ]:
def delete_batch_data(
    dataset,
    file_name_base,
    drive_base_folder,
    batch_folder,
):
    print(f"Deleting batch folders...")

    total_items = len(dataset)
    num_batches = math.ceil(total_items / batch_size)

    folder_path_batches = os.path.join(
          '/content/drive/My Drive',
          f"{drive_base_folder}/{batch_folder}"
    )

    for batch_num in range(num_batches):
        dataset_name = f"{file_name_base}_batch_{batch_num}.hf"
        batch_folder_path = os.path.join(folder_path_batches, dataset_name)

        if os.path.exists(batch_folder_path) and os.path.isdir(batch_folder_path):
            shutil.rmtree(batch_folder_path)
            print(f"Deleted folder: {batch_folder_path}")

# Process Train Dataset

In [ ]:
total_items_train = len(dataset_train)
num_batches_train = math.ceil(total_items_train / batch_size)

process_and_save_dataset(
    dataset=dataset_train,
    drive_base_folder=drive_folder,
    batch_folder = "Train Data Batches",
    agg_folder = "Train Data Agg",
    file_name_base="train_data",
    image_processor=image_processor,
    model=model,
    device=device,
    batch_size=batch_size,
    fetch_fn_kwargs=fetch_fn_kwargs
)

Processing batches:   0%|          | 0/100 [00:00<?, ?it/s]

Fetching images for batch 1/100...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Fetched 837 out of 1000 images (83.70%)
Embedding images for batch 1/100...


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Fetched 794 out of 1000 images (79.40%)
Embedding images for batch 100/100...


Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Processing batches: 100%|██████████| 100/100 [3:54:53<00:00, 140.93s/it]


Saving the dataset (0/1 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
delete_batch_data(
    dataset=dataset_train,
    file_name_base="train_data",
    drive_base_folder=drive_folder,
    batch_folder = "Train Data Batches",
)

Deleting batch folders...
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_0.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_1.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_2.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_3.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_4.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_5.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_6.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_7.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_8.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Train Data Batches/train_data_batch_9.hf
Deleted folder: /content/drive/My Drive/MIT 6.8610/Tra

# Process Validation Dataset

In [ ]:
total_items_val = len(dataset_val)
num_batches_val = math.ceil(total_items_val / batch_size)

process_and_save_dataset(
    dataset=dataset_val,
    drive_base_folder=drive_folder,
    batch_folder = "Val Data Batches",
    agg_folder = "Val Data Agg",
    file_name_base="val_data",
    image_processor=image_processor,
    model=model,
    device=device,
    batch_size=batch_size,
    fetch_fn_kwargs=fetch_fn_kwargs
)

In [ ]:
delete_batch_data(
    dataset=dataset_val,
    file_name_base="val_data",
    drive_base_folder=drive_folder,
    batch_folder = "Val Data Batches",
)

# Loading Data

## Standard Loading

In [ ]:
# from datasets import load_from_disk

# def load_dataset_from_colab(dataset_name, train_data_folder):
#     drive_base_folder = "MIT 6.8610"

#     dataset_path = os.path.join(
#             '/content/drive/My Drive',
#             f"{drive_base_folder}/{train_data_folder}/{dataset_name}"
#     )
#     return load_from_disk(dataset_path)

In [ ]:
# # Load data
# val_data_name = "val_data_10000_1700173986.82443.hf"
# train_data_name = "train_data_10000_1700173778.1307948.hf"

# val_data = load_dataset_from_colab(val_data_name, train_data_folder="Val Data Agg")
# train_data = load_dataset_from_colab(train_data_name, train_data_folder="Train Data Agg")

In [ ]:
# val_data

In [ ]:
#  train_data